
# SimSwap for videos

Reference: [my changes to the official notebook](https://gist.github.com/woctezuma/78a98b73cbba8cba478d99c8c50bc359)

## Prepare code

In [ ]:
%cd /content
!git clone https://github.com/woctezuma/SimSwap
%cd /content/SimSwap
!git checkout no-logo

In [ ]:
!pip install insightface==0.2.1 onnxruntime moviepy > /dev/null
!pip install googledrivedownloader > /dev/null
!pip install imageio==2.4.1 > /dev/null

## Prepare models

In [ ]:
%cd /content/SimSwap

In [ ]:
from google_drive_downloader import GoogleDriveDownloader

GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
                                    dest_path='./arcface_model/arcface_checkpoint.tar')

GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
                                    dest_path='./checkpoints.zip')

In [ ]:
!wget --no-check-certificate \
 https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w \
 -O antelope.zip

In [ ]:
!unzip ./checkpoints.zip  -d ./checkpoints

!unzip antelope.zip -d ./insightface_func/models/

## Prepare data

### Download

In [ ]:
%cd /content

!wget https://i.imgur.com/iQtmj1N.png -O photo.png
!wget https://i0.wp.com/john.do/wp-content/uploads/2019/07/james-franco-so-good-1.gif -O video.gif

In [ ]:
input_image_fname = '/content/photo.png'
input_video_fname = '/content/video.gif'

## Run

### Official code

In [ ]:
%cd /content/SimSwap/

In [ ]:
import cv2
import torch
import fractions
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_mutil import Face_detect_crop
from util.videoswap import video_swap
from util.add_watermark import watermark_image

In [ ]:
transformer = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

detransformer = transforms.Compose([
        transforms.Normalize([0, 0, 0], [1/0.229, 1/0.224, 1/0.225]),
        transforms.Normalize([-0.485, -0.456, -0.406], [1, 1, 1])
    ])

In [ ]:
opt = TestOptions()
opt.initialize()
opt.parser.add_argument('-f') ## dummy arg to avoid bug
opt = opt.parse()
opt.pic_a_path = input_image_fname ## or replace it with image from your own google drive
opt.video_path = input_video_fname ## or replace it with video from your own google drive
opt.output_path = './content/output.mp4'
opt.temp_path = './tmp'
opt.Arc_path = './arcface_model/arcface_checkpoint.tar'
opt.isTrain = False

crop_size = 224

torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()


app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

pic_a = opt.pic_a_path
# img_a = Image.open(pic_a).convert('RGB')
img_a_whole = cv2.imread(pic_a)
img_a_align_crop, _ = app.get(img_a_whole,crop_size)
img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0],cv2.COLOR_BGR2RGB)) 
img_a = transformer_Arcface(img_a_align_crop_pil)
img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])

# convert numpy to tensor
img_id = img_id.cuda()

#create latent id
img_id_downsample = F.interpolate(img_id, scale_factor=0.5)
latend_id = model.netArc(img_id_downsample)
latend_id = latend_id.detach().to('cpu')
latend_id = latend_id/np.linalg.norm(latend_id,axis=1,keepdims=True)
latend_id = latend_id.to('cuda')

video_swap(opt.video_path, latend_id, model, app, opt.output_path,temp_results_dir=opt.temp_path)

### My fix for GIF

If you want to apply SimSwap to a GIF, there will be an error because the input video has no audio.

To fix this issue, aggregate the temporary output by yourself by running:

In [ ]:
import os 
import cv2
import glob
from moviepy.video.io.ImageSequenceClip import ImageSequenceClip

video_path = input_video_fname
temp_results_dir = '/content/SimSwap/tmp/'
output_fname = '/content/output.gif'

video = cv2.VideoCapture(video_path)
fps = video.get(cv2.CAP_PROP_FPS)

path = os.path.join(temp_results_dir,'*.jpg')
image_filenames = sorted(glob.glob(path))

clips = ImageSequenceClip(image_filenames,fps = fps)
clips.write_gif(output_fname)

To optimize the file size (in MB) of the GIF, you can upload it to a website like https://ezgif.com/optimize